# 15-02: Bonjour/mDNS 本地服务发现使用 mDNS/Bonjour 在局域网内发现和连接服务。

In [ ]:
// 安装: npm install bonjour-service
import { Bonjour } from 'bonjour-service';
// ========== 1. 发布服务 ==========
const bonjour = new Bonjour();
// 发布一个 HTTP 服务
const service = bonjour.publish({
  name: 'My Node Server',
  type: 'http',
  port: 3000,
  txt: {
    version: '1.0.0',
    path: '/api',
    description: 'My awesome service'
  }
});
console.log('Service published:', service.name);
// 停止发布
// service.stop();

In [ ]:
// ========== 2. 发现服务 ==========
// 查找所有 HTTP 服务
const browser = bonjour.find({ type: 'http' });
browser.on('up', (service) => {
  console.log('Service up:', service.name);
  console.log('  Address:', service.addresses);
  console.log('  Port:', service.port);
  console.log('  TXT:', service.txt);
  console.log('  Host:', service.host);
});
browser.on('down', (service) => {
  console.log('Service down:', service.name);
});
// 查找特定服务
const specificBrowser = bonjour.findOne({
  type: 'http',
  name: 'My Node Server'
});
specificBrowser.on('up', (service) => {
  console.log('Found specific service:', service);
});

In [ ]:
// ========== 3. 服务发现管理器 ==========
interface DiscoveredService {
  name: string;
  type: string;
  host: string;
  port: number;
  addresses: string[];
  txt: Record<string, string>;
  discoveredAt: Date;
}
class ServiceDiscovery {
  private bonjour: Bonjour;
  private services: Map<string, DiscoveredService> = new Map();
  private browsers: any[] = [];

  constructor() {
    this.bonjour = new Bonjour();
  }

  startDiscovery(serviceType: string): void {
    const browser = this.bonjour.find({ type: serviceType });
    
    browser.on('up', (service: any) => {
      const key = `${service.name}.${service.type}`;
      const discovered: DiscoveredService = {
        name: service.name,
        type: service.type,
        host: service.host,
        port: service.port,
        addresses: service.addresses,
        txt: service.txt,
        discoveredAt: new Date()
      };
      
      this.services.set(key, discovered);
      this.emit('serviceUp', discovered);
    });
    
    browser.on('down', (service: any) => {
      const key = `${service.name}.${service.type}`;
      const discovered = this.services.get(key);
      if (discovered) {
        this.services.delete(key);
        this.emit('serviceDown', discovered);
      }
    });
    
    this.browsers.push(browser);
  }

  getServices(): DiscoveredService[] {
    return Array.from(this.services.values());
  }

  getServiceByName(name: string): DiscoveredService | undefined {
    return Array.from(this.services.values()).find(s => s.name === name);
  }

  stop(): void {
    this.browsers.forEach(b => b.stop());
    this.bonjour.destroy();
  }

  private emit(event: string, data: any): void {
    // 简化版事件发射
    console.log(`[${event}]`, data.name);
  }
}
// 使用
// const discovery = new ServiceDiscovery();
// discovery.startDiscovery('http');
// console.log(discovery.getServices());

In [ ]:
// ========== 4. 局域网通信 ==========
import { createServer, connect } from 'net';
// TCP 服务端
class LANServer {
  private server: ReturnType<typeof createServer>;
  private bonjour: Bonjour;
  private service: any;

  constructor(private port: number, private name: string) {
    this.bonjour = new Bonjour();
    this.server = createServer((socket) => {
      console.log('Client connected:', socket.remoteAddress);
      
      socket.on('data', (data) => {
        console.log('Received:', data.toString());
        socket.write(`Echo: ${data}`);
      });
      
      socket.on('end', () => {
        console.log('Client disconnected');
      });
    });
  }

  start(): void {
    this.server.listen(this.port, () => {
      console.log(`Server listening on port ${this.port}`);
      
      // 发布服务
      this.service = this.bonjour.publish({
        name: this.name,
        type: 'myapp',
        port: this.port
      });
    });
  }

  stop(): void {
    this.service?.stop();
    this.server.close();
    this.bonjour.destroy();
  }
}
// TCP 客户端
class LANClient {
  private bonjour: Bonjour;

  constructor() {
    this.bonjour = new Bonjour();
  }

  async findAndConnect(serviceName: string): Promise<void> {
    return new Promise((resolve, reject) => {
      const browser = this.bonjour.findOne({ type: 'myapp' });
      
      browser.on('up', (service: any) => {
        if (service.name === serviceName) {
          this.connect(service.host, service.port);
          resolve();
        }
      });
      
      // 超时
      setTimeout(() => {
        reject(new Error('Service discovery timeout'));
      }, 10000);
    });
  }

  private connect(host: string, port: number): void {
    const client = connect(port, host, () => {
      console.log('Connected to server');
      client.write('Hello from client!');
    });
    
    client.on('data', (data) => {
      console.log('Received:', data.toString());
    });
    
    client.on('end', () => {
      console.log('Disconnected from server');
    });
  }

  stop(): void {
    this.bonjour.destroy();
  }
}

In [ ]:
// ========== 5. WebSocket over LAN ==========
// 使用 mDNS 发现 WebSocket 服务
import { WebSocketServer, WebSocket } from 'ws';
class WebSocketDiscovery {
  private wss: WebSocketServer | null = null;
  private bonjour: Bonjour;
  private service: any;

  constructor() {
    this.bonjour = new Bonjour();
  }

  startServer(port: number): void {
    this.wss = new WebSocketServer({ port });
    
    this.wss.on('connection', (ws) => {
      console.log('WebSocket client connected');
      
      ws.on('message', (message) => {
        console.log('Received:', message.toString());
        ws.send(`Echo: ${message}`);
      });
    });
    
    // 发布服务
    this.service = this.bonjour.publish({
      name: 'WebSocket Server',
      type: 'ws',
      port,
      txt: { path: '/' }
    });
    
    console.log(`WebSocket server started on port ${port}`);
  }

  async connectToServer(serverName: string): Promise<WebSocket> {
    return new Promise((resolve, reject) => {
      const browser = this.bonjour.findOne({ type: 'ws' });
      
      browser.on('up', (service: any) => {
        if (service.name === serverName) {
          const ws = new WebSocket(`ws://${service.host}:${service.port}`);
          ws.on('open', () => resolve(ws));
          ws.on('error', reject);
        }
      });
      
      setTimeout(() => reject(new Error('Discovery timeout')), 10000);
    });
  }

  stop(): void {
    this.wss?.close();
    this.service?.stop();
    this.bonjour.destroy();
  }
}